## PyTorch Tensors

In [71]:

import torch

In [72]:



dtype = torch.float
device = torch.device("cpu")

In [73]:
N,D_in,H,D_out=64,1000,100,10

In [74]:
x = torch.randn(N,D_in, device=device,dtype=dtype)
y = torch.randn(N,D_out, device=device,dtype=dtype)

In [75]:
w1=torch.randn(D_in,H,device=device,dtype=dtype)
w2=torch.randn(H,D_out,device=device,dtype=dtype)

In [76]:
learning_rate=1e-6

In [77]:
for t in range(500):
    h=x.mm(w1)
    h_relu=h.clamp(min=0)
    y_pred=h_relu.mm(w2)

In [78]:
loss=(y_pred-y).pow(2).sum().item()
if t % 100==99:
    print(t,loss)

499 30792762.0


In [79]:
grad_y_pred = 2.0 * (y_pred - y)
grad_w2 = h_relu.t().mm(grad_y_pred)
grad_h_relu = grad_y_pred.mm(w2.t())
grad_h = grad_h_relu.clone()
grad_h[h < 0] = 0
grad_w1 = x.t().mm(grad_h)

In [88]:
w1 -= learning_rate * grad_w1
w2 -= learning_rate * grad_w2

In [139]:
import torch
dtype=torch.float
device=torch.device("cpu")
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)


learning_rate=1e-6

for t in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()
   

99 632.7509155273438
199 3.6523475646972656
299 0.034113809466362
399 0.0006434448296204209
499 7.068233389873058e-05


# NN MODULE

In [146]:
import torch

N,D_in,H,D_out=64,1000,100,10
x=torch.randn(N,D_in)
y=torch.randn(N,D_out)

model=torch.nn.Sequential(
      torch.nn.Linear(D_in,H),
      torch.nn.ReLU(),
      torch.nn.Linear(H,D_out)
)

loss_fn=torch.nn.MSELoss(reduction="sum")
learning_rate=1e-4
for t in range(500):
    y_pred=model(x)
    loss=loss_fn(y_pred,y)
    if t%100==99:
        print(t,loss.item())
        
    model.zero_grad()
    
    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -=learning_rate*param.grad

99 2.9755284786224365
199 0.050731685012578964
299 0.0012980748433619738
399 3.967877273680642e-05
499 1.3843666692991974e-06


# PyTorch:OPTIM

In [160]:
import torch
N,D_in,H,D_out=64,1000,100,10

x=torch.randn(N,D_in)
y=torch.randn(N,D_out)

model=torch.nn.Sequential(
      torch.nn.Linear(D_in,H),
      torch.nn.ReLU(),
      torch.nn.Linear(H,D_out),
)

loss_fn=torch.nn.MSELoss(reduction="sum")
learning_rate=1e-4
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
for t in range(500):
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t% 100 == 99:
        print(t,   loss.item())
        
        
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        

99 40.9500846862793
199 0.4600886106491089
299 0.012946339324116707
399 0.00024288073473144323
499 1.5488204780922388e-06


# DEFINING NEW AUTOGRAD FUNCTIONS

In [181]:
import torch
class MyReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx,input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    @staticmethod
    def backward(ctx,grad_output):
        input,=ctx.saved_tensors
        grad_input=grad_output.clone()
        grad_input[input<0]=0
        return grad_input
        
dtype=torch.float
device=torch.device("cpu")
N,D_in,H,D_out=64,1000,100,10

x=torch.randn(N,D_in,device=device,dtype=dtype)
y=torch.randn(N,D_out,device=device,dtype=dtype)

w1=torch.randn(D_in,H,device=device,dtype=dtype,requires_grad=True)
w2=torch.randn(H,D_out,device=device,dtype=dtype,requires_grad=True)

learning_rate=1e-6
for t in range(500):
    relu=MyReLU.apply
    y_pred = relu(x.mm(w1)).mm(w2)
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()
    
    with torch.no_grad():
        w1 -= learning_rate*w1.grad
        w2 -= learning_rate*w2.grad

99 65159184.0
199 nan
299 nan
399 nan
499 nan


In [176]:
import torch


class MyReLU(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # To apply our Function, we use Function.apply method. We alias this as 'relu'.
    relu = MyReLU.apply

    # Forward pass: compute predicted y using operations; we compute
    # ReLU using our custom autograd operation.
    y_pred = relu(x.mm(w1)).mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

99 745.9379272460938
199 5.555403709411621
299 0.06596189737319946
399 0.0011977539397776127
499 0.00010636385559337214
